In [ ]:
import metmhn.regularized_optimization as reg_opt
import metmhn.Utilityfunctions as utils

import pandas as pd
import warnings
warnings.simplefilter(action='ignore', 
                      category=pd.errors.PerformanceWarning)

import jax.numpy as jnp
import numpy as np
import jax as jax
jax.config.update("jax_enable_x64", True)
import matplotlib.pyplot as plt
import logging
# Adapt path to where logs should be kept
logging.basicConfig(filename='../logs/analysis_example.log',
                    format='%(asctime)s %(levelname)-8s %(message)s',
                    filemode='w', 
                    level=logging.INFO, 
                    force=True,
                    datefmt='%Y-%m-%d %H:%M:%S'
                    )

In the following example, we have information about the genotypes of Primary Tumors (PTs) and Metastases (MTs) for patients suffering from lung adenocarcinomas (LUADs). \
For some patients we only know the status of either the PT or the MT. This is indicated in our data in the column 'isPaired'. \
Additionally, the column 'metaStatus' indicates wether a metastasis was observed or not ('present', 'isMetastasis', 'absent'). \
This dataset contains mutations as well as copy number alterations. For this example we restrict this dataset to only contain mutations, such that it runs on average desktops on cpus only in ~20 minutes.

In [ ]:
mut_handle = "../data/luad/G14_LUAD_Events.csv"
annot_handle = "../data/luad/G14_LUAD_sampleSelection.csv"
annot_data = pd.read_csv(annot_handle)
mut_data = pd.read_csv(mut_handle)
mut_data.rename(columns={"Unnamed: 0":"patientID"}, inplace = True)
dat = pd.merge(mut_data, annot_data.loc[:, ['patientID', 'metaStatus']], 
               on=["patientID", "patientID"])
muts = ['P.TP53 (M)', 'M.TP53 (M)', 'P.KRAS (M)', 'M.KRAS (M)', 'P.EGFR (M)', 'M.EGFR (M)', 'P.STK11 (M)', 'M.STK11 (M)', 'P.KEAP1 (M)', 'M.KEAP1 (M)', 
        'P.RBM10 (M)', 'M.RBM10 (M)', 'P.SMARCA4 (M)', 'M.SMARCA4 (M)', 'P.ATM (M)', 'M.ATM (M)', 'P.NF1 (M)', 'M.NF1 (M)', 'P.PTPRD (M)', 'M.PTPRD (M)', 
        'P.PTPRT (M)', 'M.PTPRT (M)', 'P.ARID1A (M)', 'M.ARID1A (M)', 'P.BRAF (M)', 'M.BRAF (M)', 'P.PIK3CA (M)', 'M.PIK3CA (M)', 'P.EPHA3 (M)', 'M.EPHA3 (M)', 
        'P.FAT1 (M)', 'M.FAT1 (M)', 'P.SETD2 (M)', 'M.SETD2 (M)', 'P.RB1 (M)', 'M.RB1 (M)', 'P.MET (M)', 'M.MET (M)', 'P.KMT2C (M)', 'M.KMT2C (M)']

Next, we convert string labels to numeric labels and remove corrupted datapoints. Finally, we convert the resulting dataframe to a jax.numpy array.

In [ ]:
# Label each datapoint with a numeric value according to its sequencetype
dat["type"] = dat.apply(utils.categorize, axis=1)
dat["Seeding"] = dat.apply(utils.add_seeding, axis=1)
dat.loc[dat["M.AgeAtSeqRep"] == "No metastasis included", "M.AgeAtSeqRep"] = pd.NA
dat.loc[dat["P.AgeAtSeqRep"] == "No primary included", "P.AgeAtSeqRep"] = pd.NA
dat["P.AgeAtSeqRep"] = dat["P.AgeAtSeqRep"].astype(pd.Int64Dtype())
dat["M.AgeAtSeqRep"] = dat["M.AgeAtSeqRep"].astype(pd.Int64Dtype())
dat["diag_order"] = dat["M.AgeAtSeqRep"] - dat["P.AgeAtSeqRep"]
dat["diag_order"] = dat["diag_order"].apply(lambda x: pd.NA if pd.isna(x) else 2 if x < 0 else 1 if x > 0 else 0) 
dat["diag_order"] = dat["diag_order"].astype(pd.Int64Dtype())
events_data = muts+["Seeding"]
# Only use datapoints where the state of the metastasis is known
cleaned = dat.loc[dat["type"].isin([0,1,2,3]), muts+["Seeding", "diag_order", "type"]]
# Remove completely empty datapoints
cleaned.drop(cleaned[cleaned.iloc[:,:-2].sum(axis=1)<1].index, inplace=True)
dat = jnp.array(cleaned.to_numpy(dtype=np.int8, na_value=-99))

Retrieve the event names and trim the PT/MT identifier:

In [ ]:
events_plot = []
for elem in cleaned.columns[:-3].to_list()[::2]:
    full_mut_id = elem.split(".")
    events_plot.append(full_mut_id[1])
events_plot.append("Seeding")

Enumerate the frequencies of SNVs and CNVs in all subgroups. 'NM/EM' refer to Never Metastasizing/ Ever Metastasizing tumors, where only a single genotype is known. A Mutation is referred to as 'MT/PT-private' if happens exclusively in the MT or PT, otherwise it is called 'shared':

In [ ]:
n_tot = (cleaned.shape[1]-1)//2 + 1
n_mut = n_tot-1
utils.marg_frequs(dat, events_plot)

Optional: We use a  sparsity and symmetry promoting penalty. The weight of the penalization can be determined in a k-fold crossvalidation. Note: Running the full 5-fold crossvalidation across 5 splits below takes around 2 hours on an average Desktop-CPU.

In [ ]:
log_lams = np.linspace(-3.5, -2.5, 5)
m_p_corr = 0.65
lams = 10**log_lams
penal_weight, se = utils.cross_val(dat=dat, 
                               penal_fun=reg_opt.symmetric_penal, 
                               splits=lams, 
                               n_folds=5, 
                               m_p_corr=m_p_corr, 
                               seed=42)

Afterwards we can train an metMHN-model. The example below takes around 10 minutes to finish on average Desktop-CPU:

In [ ]:
th_init, dp_init, dm_init = utils.indep(dat)
theta, d_p, d_m= reg_opt.learn_mhn(th_init=th_init, 
                                   dp_init=dp_init,
                                   dm_init=dm_init,
                                   dat=dat,
                                   perc_met=m_p_corr,
                                   penal=reg_opt.symmetric_penal,
                                   w_penal=penal_weight,
                                   opt_ftol=1e-05
                                   )

Visualize the results:

In [ ]:
th_plot = np.row_stack((d_p.reshape((1,-1)), 
                        d_m.reshape((1,-1)), 
                        theta))
fig, (ax1, ax2) = plt.subplots(1,2, sharey="col", figsize=(14,9),
                                gridspec_kw={'width_ratios': [n_tot, 1], "wspace": -0.6})
utils.plot_theta(ax1, ax2, th_plot, events_plot, alpha=0.05, font_size=5)
plt.show()

In [ ]:
df2 = pd.DataFrame(th_plot, columns=events_plot)
df2.to_csv("../results/luad/luad_g14_20muts.csv")